In [7]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from tqdm import tqdm

def croll_page(driver, html):
    soup = BeautifulSoup(html, 'html.parser')
    store_items = soup.select('.PlaceItem.clickArea')

    for store in tqdm(store_items):
        score_blind = store.select_one('[data-id="scoreBlind"]')
        if score_blind and 'HIDDEN' not in score_blind.get('class', ''):  # HIDDEN이 없는 경우 스킵
            continue
        
        store_name_element = store.select_one('.head_item > .tit_name > .link_name')
        store_name = store_name_element.text.strip() if store_name_element else '이름 없음'
        
        try:
            name_element = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.LINK_TEXT, store_name)))
            driver.execute_script("arguments[0].click();", name_element)
            time.sleep(1)
            
            place_links = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[data-id="name"]')))
            
            for link in place_links:
                href = link.get_attribute("href")
                if href and "none" not in href:
                    print(f"{store_name} 링크: {href}")
                    driver.execute_script(f"window.open('{href}');")
                    time.sleep(1)
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(1)

                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')

                    st_name = soup.find('h3', class_='tit_place')
                    st_name = st_name.get_text(strip=True).replace('장소명', '') if st_name else '이름 없음'
                    
                    rating = soup.find('span', class_='num_star')
                    rating = rating.get_text(strip=True) if rating else '평점 없음'
                    
                    reviews = soup.find('a', class_='link_review')
                    reviews = reviews.get_text(strip=True) if reviews else '후기 없음'
                    
                    blog_reviews = soup.find('a', class_='link_blog')
                    blog_reviews = blog_reviews.get_text(strip=True) if blog_reviews else '블로그 후기 없음'
                    
                    category_element = soup.find('span', class_='info_cate')
                    category = category_element.get_text(strip=True) if category_element else '카테고리 없음'
                    
                    address_element = soup.find('span', class_='txt_detail')
                    address = address_element.get_text(strip=True) if address_element else '주소 없음'
                    
                    print(f'장소명: {st_name}')
                    print(f'별점: {rating}')
                    print(f'후기 개수: {reviews}')
                    print(f'블로그 후기 개수: {blog_reviews}')
                    print(f'카테고리: {category}')
                    print(f'주소: {address}')
                    
                    driver.close()
                    time.sleep(1)
                    driver.switch_to.window(driver.window_handles[0])
                    break
        except Exception as e:
            print(f"{store_name} 클릭 실패: {e}")

# 카카오맵 URL
URL = 'https://map.kakao.com/'
KEYWORD = '가산동 맛집'

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')

driver = webdriver.Chrome(options=options)
driver.get(URL)

search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]')
search_area.send_keys(KEYWORD)
driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)
time.sleep(1)

driver.find_element(By.XPATH, '//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)
time.sleep(1)

j = 0
while True:
    num_pages = 5
    for i in range(num_pages):
        print("page" + str((i+1) + 5*j))
        page_section = driver.find_element(By.ID, 'info.search.page')
        page_elements = page_section.find_elements(By.TAG_NAME, 'a')
        page = page_elements[i]
        driver.execute_script("arguments[0].click();", page)
        time.sleep(1)
        html = driver.page_source
        croll_page(driver, html)
    
    next_button = driver.find_element(By.ID, 'info.search.page.next')
    if 'disabled' in next_button.get_attribute('class'):
        print('다음 페이지 없음. 종료')
        break
    else:
        j += 1
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'info.search.page')))  # 페이지 로딩 대기

driver.quit()


page1


  0%|          | 0/15 [00:00<?, ?it/s]

춘천옥 링크: https://place.map.kakao.com/8117410
장소명: 이름 없음
별점: 평점 없음
후기 개수: 후기 없음
블로그 후기 개수: 블로그 후기 없음
카테고리: 카테고리 없음
주소: 주소 없음


  7%|▋         | 1/15 [00:04<01:03,  4.54s/it]

삼숙이갈비 본점 링크: https://place.map.kakao.com/26943942
장소명: 삼숙이갈비 본점
별점: 4.1
후기 개수: 후기16개
블로그 후기 개수: 블로그74개
카테고리: 장소 카테고리갈비
주소: 서울 금천구 벚꽃로 308 1층 (우)08509


...

 60%|██████    | 9/15 [00:40<00:27,  4.55s/it]

휴스시 링크: https://place.map.kakao.com/1919628439
장소명: 휴스시
별점: 4.3
후기 개수: 후기35개
블로그 후기 개수: 블로그36개
카테고리: 장소 카테고리초밥,롤
주소: 서울 금천구 디지털로9길 46 이앤씨드림타워7차 1층 108호 (우)08512


100%|██████████| 15/15 [00:45<00:00,  3.03s/it]

휴스시 클릭 실패: list index out of range
미가양꼬치 클릭 실패: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.165)
Stacktrace:
#0 0x59cbe9bfeffa <unknown>
#1 0x59cbe96bd970 <unknown>
#2 0x59cbe9691c45 <unknown>
#3 0x59cbe973da3e <unknown>
#4 0x59cbe975b09c <unknown>
#5 0x59cbe9735063 <unknown>
#6 0x59cbe9701328 <unknown>
#7 0x59cbe9702491 <unknown>
#8 0x59cbe9bc642b <unknown>
#9 0x59cbe9bca2ec <unknown>
#10 0x59cbe9bada22 <unknown>
#11 0x59cbe9bcae64 <unknown>
#12 0x59cbe9b91bef <unknown>
#13 0x59cbe9bed558 <unknown>
#14 0x59cbe9bed736 <unknown>
#15 0x59cbe9bfde76 <unknown>
#16 0x72e6f2294ac3 <unknown>

오늘은 초밥먹는날 클릭 실패: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.165)
Stacktrace:
#0 0x59cbe9bfeffa <unknown>
#1 0x59cbe96bd970 <unknown>
#2 0x59cbe9691c45 <unknown>
#3 0x59cbe973da3e <unknown>
#4 0x59cbe975b09c <unknown>
#5 0x59cbe9735063 <un

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.165)
Stacktrace:
#0 0x59cbe9bfeffa <unknown>
#1 0x59cbe96bd970 <unknown>
#2 0x59cbe9691c45 <unknown>
#3 0x59cbe973da3e <unknown>
#4 0x59cbe975b09c <unknown>
#5 0x59cbe9735063 <unknown>
#6 0x59cbe9701328 <unknown>
#7 0x59cbe9702491 <unknown>
#8 0x59cbe9bc642b <unknown>
#9 0x59cbe9bca2ec <unknown>
#10 0x59cbe9bada22 <unknown>
#11 0x59cbe9bcae64 <unknown>
#12 0x59cbe9b91bef <unknown>
#13 0x59cbe9bed558 <unknown>
#14 0x59cbe9bed736 <unknown>
#15 0x59cbe9bfde76 <unknown>
#16 0x72e6f2294ac3 <unknown>
